<a href="https://colab.research.google.com/github/Juni-Choi/web/blob/master/Twitter_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트위터 날씨봇 만들기


## 목적 : -> 트위터에 매일 '오늘의 날씨' 자동 업로드

![Reporter](./report.png)

트위터봇 예시 :  https://namu.wiki/w/%ED%8A%B8%EC%9C%84%ED%84%B0%20%EB%B4%87/%EB%AA%A9%EB%A1%9D

트위터 날씨봇 링크 : https://twitter.com/epower_marathon

# 날씨 데이터 가져오기

참조 : https://it-neicebee.tistory.com/48

In [1]:
def get_weather_data(location = "오산시 궐동"):
    
    import requests 
    from bs4 import BeautifulSoup
    #지역을 입력하지 않을 경우 직접 물어보기
    if location == "":
        location = input("지역을 입력하세요\n>>> ") 
        
    Finallocation = location + '날씨' 
    LocationInfo = "" 
    NowTemp = "" 
    CheckDust = [] 
    url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=' + Finallocation 
    hdr = {'User-Agent': ('mozilla/5.0 (windows nt 10.0; win64; x64) applewebkit/537.36 (khtml, like gecko) chrome/78.0.3904.70 safari/537.36')} 
    req = requests.get(url, headers=hdr) 
    html = req.text 
    soup = BeautifulSoup(html, 'html.parser') 
    # 오류 체크 
    ErrorCheck = soup.find('span', {'class' : 'btn_select'}) 
    if 'None' in str(ErrorCheck): 
        print("Error! 지역 검색 오류!") 
    else: 
        # 지역 정보 
        for i in soup.select('span[class=btn_select]'): 
            LocationInfo = i.text
    # 현재 온도 
    NowTemp = soup.find('span', {'class': 'todaytemp'}).text + soup.find('span', {'class' : 'tempmark'}).text[2:] 
    # 날씨 캐스트 
    WeatherCast = soup.find('p', {'class' : 'cast_txt'}).text 
    # 오늘 오전온도, 오후온도, 체감온도 
    TodayMorningTemp = soup.find('span', {'class' : 'min'}).text 
    TodayAfternoonTemp = soup.find('span', {'class' : 'max'}).text 
    TodayFeelTemp = soup.find('span', {'class' : 'sensible'}).text[5:] 
    # 자외선 지수 
    TodayUV = soup.find('span', {'class' : 'indicator'}).text[4:-2] + " " + soup.find('span', {'class' : 'indicator'}).text[-2:] 
    # 미세먼지, 초미세먼지, 오존 지수 
    CheckDust1 = soup.find('div', {'class': 'sub_info'}) 
    CheckDust2 = CheckDust1.find('div', {'class': 'detail_box'}) 
    for i in CheckDust2.select('dd'): 
        CheckDust.append(i.text) 

    FineDust = CheckDust[0][:-2] + " " + CheckDust[0][-2:] 
    UltraFineDust = CheckDust[1][:-2] + " " + CheckDust[1][-2:] 
    Ozon = CheckDust[2][:-2] + " " + CheckDust[2][-2:] 

    # 내일 오전, 오후 온도 및 상태 체크 
    tomorrowArea = soup.find('div', {'class': 'tomorrow_area'}) 
    tomorrowCheck = tomorrowArea.find_all('div', {'class': 'main_info morning_box'}) 
    # 내일 오전온도 
    tomorrowMoring1 = tomorrowCheck[0].find('span', {'class': 'todaytemp'}).text 
    tomorrowMoring2 = tomorrowCheck[0].find('span', {'class' : 'tempmark'}).text[2:] 
    tomorrowMoring = tomorrowMoring1 + tomorrowMoring2 
    # 내일 오전상태 
    tomorrowMState1 = tomorrowCheck[0].find('div', {'class' : 'info_data'}) 
    tomorrowMState2 = tomorrowMState1.find('ul', {'class' : 'info_list'}) 
    tomorrowMState3 = tomorrowMState2.find('p', {'class' : 'cast_txt'}).text 
    tomorrowMState4 = tomorrowMState2.find('div', {'class' : 'detail_box'}) 
    tomorrowMState5 = tomorrowMState4.find('span').text.strip() 
    tomorrowMState = tomorrowMState3 + " " + tomorrowMState5 
    # 내일 오후온도 
    tomorrowAfter1 = tomorrowCheck[1].find('p', {'class' : 'info_temperature'}) 
    tomorrowAfter2 = tomorrowAfter1.find('span', {'class' : 'todaytemp'}).text
    tomorrowAfter3 = tomorrowAfter1.find('span', {'class' : 'tempmark'}).text[2:] 
    tomorrowAfter = tomorrowAfter2 + tomorrowAfter3 
    # 내일 오후상태 
    tomorrowAState1 = tomorrowCheck[1].find('div', {'class' : 'info_data'}) 
    tomorrowAState2 = tomorrowAState1.find('ul', {'class' : 'info_list'}) 
    tomorrowAState3 = tomorrowAState2.find('p', {'class' : 'cast_txt'}).text 
    tomorrowAState4 = tomorrowAState2.find('div', {'class' : 'detail_box'}) 
    tomorrowAState5 = tomorrowAState4.find('span').text.strip() 
    tomorrowAState = tomorrowAState3 + " " + tomorrowAState5 

    print("=========================================") 
    print(LocationInfo + " 날씨 정보입니다.") 
    print("=========================================") 
    print("현재온도: " + NowTemp) 
    print("체감온도: " + TodayFeelTemp) 
    print("오전/오후 온도: " + TodayMorningTemp + "/" + TodayAfternoonTemp) 
    print("현재 상태: " + WeatherCast) 
    print("현재 자외선 지수: " + TodayUV) 
    print("현재 미세먼지 농도: " + FineDust) 
    print("현재 초미세먼지 농도: " + UltraFineDust) 
    print("현재 오존 지수: " + Ozon) 
    print("=========================================") 
    print(LocationInfo + " 내일 날씨 정보입니다.") 
    print("=========================================") 
    print("내일 오전 온도: " + tomorrowMoring) 
    print("내일 오전 상태: " + tomorrowMState) 
    print("내일 오후 온도: " + tomorrowAfter) 
    print("내일 오후 상태: " + tomorrowAState)
    
    #위 내용을 텍스트로 가져오는걸 직접 보여드리겠습니다!


In [ ]:
get_weather_data()

In [ ]:
get_weather_data("대전광역시 유성구 지족동")

# 자! 위는 출력만 하는 함수인데 이제 저 텍스트를 가져오려면 어떻게 해야 할까요?

## hint : return

# 과제 :  시간 : 30분 (코딩이 가능하신 분들만)

- 위 코드에서 
 - 현재 온도, 
 - 체감온도, 
 - 미세먼지 농도, 
 - 현재 초미세먼지 농도, 
 - 현재 미세먼지 농도
 - 내일 오전 온도,
 - 내일 오전 상태 
 - 내일 오후 온도 
 - 내일 오후 상태
 
- 만 추려서 나만의 날씨 정보를 만들어보세요! 멘트는 자유롭게 하시고 저에게 보여주시면 됩니다!
-

# 과제 for 코딩이 어려우신 분들
- 코딩이 어려우신 분들은 앞의 스케쥴링 과제로 돌아가셔서, 위 코드를 실행하시고, 매 분 마다 크롤링 한 내용을 출력할 수 있는 코드를 한번 작성해보세요!

In [6]:
def job():
  print("I'm Working...")

In [10]:
def get_today_weather():
  return "sunny"

In [13]:
today_weather = get_today_weather()
print(today_weather)

sunny


# 텍스트 가져오기 시연

In [111]:
def get_weather_data1(location = "오산시 궐동"):
    
    import requests 
    from bs4 import BeautifulSoup
    #지역을 입력하지 않을 경우 직접 물어보기
    if location == "":
        location = input("지역을 입력하세요\n>>> ") 
        
    Finallocation = location + '날씨' 
    LocationInfo = "" 
    NowTemp = "" 
    CheckDust = [] 
    url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=' + Finallocation 
    hdr = {'User-Agent': ('mozilla/5.0 (windows nt 10.0; win64; x64) applewebkit/537.36 (khtml, like gecko) chrome/78.0.3904.70 safari/537.36')} 
    req = requests.get(url, headers=hdr) 
    html = req.text 
    soup = BeautifulSoup(html, 'html.parser') 
    # 오류 체크 
    ErrorCheck = soup.find('span', {'class' : 'btn_select'}) 
    if 'None' in str(ErrorCheck): 
        print("Error! 지역 검색 오류!") 
    else: 
        # 지역 정보 
        for i in soup.select('span[class=btn_select]'): 
            LocationInfo = i.text
    # 현재 온도 
    NowTemp = soup.find('span', {'class': 'todaytemp'}).text + soup.find('span', {'class' : 'tempmark'}).text[2:] 
    # 날씨 캐스트 
    WeatherCast = soup.find('p', {'class' : 'cast_txt'}).text 
    # 오늘 오전온도, 오후온도, 체감온도 
    TodayMorningTemp = soup.find('span', {'class' : 'min'}).text 
    TodayAfternoonTemp = soup.find('span', {'class' : 'max'}).text 
    TodayFeelTemp = soup.find('span', {'class' : 'sensible'}).text[5:] 
    # 자외선 지수 
    TodayUV = soup.find('span', {'class' : 'indicator'}).text[4:-2] + " " + soup.find('span', {'class' : 'indicator'}).text[-2:] 
    # 미세먼지, 초미세먼지, 오존 지수 
    CheckDust1 = soup.find('div', {'class': 'sub_info'}) 
    CheckDust2 = CheckDust1.find('div', {'class': 'detail_box'}) 
    for i in CheckDust2.select('dd'): 
        CheckDust.append(i.text) 

    FineDust = CheckDust[0][:-2] + " " + CheckDust[0][-2:] 
    UltraFineDust = CheckDust[1][:-2] + " " + CheckDust[1][-2:] 
    Ozon = CheckDust[2][:-2] + " " + CheckDust[2][-2:] 

    # 내일 오전, 오후 온도 및 상태 체크 
    tomorrowArea = soup.find('div', {'class': 'tomorrow_area'}) 
    tomorrowCheck = tomorrowArea.find_all('div', {'class': 'main_info morning_box'}) 
    # 내일 오전온도 
    tomorrowMoring1 = tomorrowCheck[0].find('span', {'class': 'todaytemp'}).text 
    tomorrowMoring2 = tomorrowCheck[0].find('span', {'class' : 'tempmark'}).text[2:] 
    tomorrowMoring = tomorrowMoring1 + tomorrowMoring2 
    # 내일 오전상태 
    tomorrowMState1 = tomorrowCheck[0].find('div', {'class' : 'info_data'}) 
    tomorrowMState2 = tomorrowMState1.find('ul', {'class' : 'info_list'}) 
    tomorrowMState3 = tomorrowMState2.find('p', {'class' : 'cast_txt'}).text 
    tomorrowMState4 = tomorrowMState2.find('div', {'class' : 'detail_box'}) 
    tomorrowMState5 = tomorrowMState4.find('span').text.strip() 
    tomorrowMState = tomorrowMState3 + " " + tomorrowMState5 
    # 내일 오후온도 
    tomorrowAfter1 = tomorrowCheck[1].find('p', {'class' : 'info_temperature'}) 
    tomorrowAfter2 = tomorrowAfter1.find('span', {'class' : 'todaytemp'}).text
    tomorrowAfter3 = tomorrowAfter1.find('span', {'class' : 'tempmark'}).text[2:] 
    tomorrowAfter = tomorrowAfter2 + tomorrowAfter3 
    # 내일 오후상태 
    tomorrowAState1 = tomorrowCheck[1].find('div', {'class' : 'info_data'}) 
    tomorrowAState2 = tomorrowAState1.find('ul', {'class' : 'info_list'}) 
    tomorrowAState3 = tomorrowAState2.find('p', {'class' : 'cast_txt'}).text 
    tomorrowAState4 = tomorrowAState2.find('div', {'class' : 'detail_box'}) 
    tomorrowAState5 = tomorrowAState4.find('span').text.strip() 
    tomorrowAState = tomorrowAState3 + " " + tomorrowAState5 

    final_text = """{LocationInfo}의 오늘 날씨는 오전 {TodayMorningTemp}, 오후 {TodayAfternoonTemp}, 체감온도 {TodayFeelTemp}, 미세먼지 {FineDust}, 초미세먼지 {UltraFineDust}, 오존농도 {Ozon}, 자외선지수 {TodayUV}입니다.
    """.format(LocationInfo = LocationInfo,TodayMorningTemp =TodayMorningTemp, TodayAfternoonTemp = TodayAfternoonTemp, TodayFeelTemp = TodayFeelTemp, FineDust = FineDust, UltraFineDust = UltraFineDust, Ozon = Ozon, TodayUV = TodayUV) 
           # tomorrowMoring = tomorrowMoring, tomorrowAfter = tomorrowAfter, tomorrowAState = tomorrowAState)


    return final_text
   
    #위 내용을 텍스트로 가져오는걸 직접 보여드리겠습니다!


In [112]:
get_weather_data1("오산시 궐동")

'경기도 오산시 궐동의 오늘 날씨는 오전 14˚, 오후 24˚, 체감온도 23.4˚, 미세먼지 12㎍/㎥ 좋음, 초미세먼지 1㎍/㎥ 좋음, 오존농도 0.033ppm 보통, 자외선지수 5 보통입니다.\n    '

In [1]:
NowTemp = 23
TodayFeelTemp = 4
final_text = "현재 온도 : " + str(NowTemp) + "\n" + \
              "TodayfinalTemp " + str(TodayFeelTemp)
final_text

'현재 온도 : 23\nTodayfinalTemp 4'

In [45]:
pip install tweepy

# 트위터 봇 만들기 시작!

# twitter api를 활용한 twitter upload

# twitter api 발급 받기

- 트위터 계정을 프로그램 개발에 사용할 수 있도록 제 notion 링크 참조

- 발급 받으면 consumer_key, consumer_secret, access_token, access_token_secret 가 주어짐.

#### anaconda prompt에 필요한 패키지 설치

pip install tweepy

In [87]:
# 트위터 api 사용 모듈 가져오기
import tweepy

In [88]:
# 인증키값들 써주기
#oetgo@daum.net
#API Key 
consumer_key = 'ExcRL3OPMPwVVOTx2VXNUvrQY'
#API Key_Secret
consumer_secret = 'wolrZNwPxIqqH9SJhZVjbrRFhzut3N388VKS7YCzgRdYuTog45'
access_token = '2769145560-kLzZ2BSqj0xpwAUkS6aqBQ2qk6FgXVaamgWHXCZ'
access_token_secret = 'IE8x6b5jqpZKF5DZspUHwaosP8k4iakEORAiwkGNhRJ4w'

In [89]:
# 인증키값들 보내주기
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) #consumer_key, consumer_secret
auth.set_access_token(access_token, access_token_secret) #access_token, access_token_secret
api = tweepy.API(auth)
api

In [108]:
# 업로드할 내용 써주기
tweet = '안녕하세요! Juni 입니다.\n ' + get_weather_data1() # 내 이름 + 문구 써주기
tweet

'안녕하세요! Juni 입니다.\n 경기도 오산시 궐동의 날씨는 오전 14˚, 오후 24˚, 체감온도 23.2˚, 미세먼지 12㎍/㎥ 좋음, 초미세먼지 1㎍/㎥ 좋음, 오존농도 0.033ppm 보통, 자외선지수 5 보통입니다.\n    '

In [109]:
# 트윗 업로드!
api.update_status(status=tweet)

Status(_api=<tweepy.api.API object at 0x7f54d00bf3c8>, _json={'created_at': 'Thu Sep 24 09:18:51 +0000 2020', 'id': 1309059713540259840, 'id_str': '1309059713540259840', 'text': '안녕하세요! Juni 입니다.\n 경기도 오산시 궐동의 날씨는 오전 14˚, 오후 24˚, 체감온도 23.2˚, 미세먼지 12㎍/㎥ 좋음, 초미세먼지 1㎍/㎥ 좋음, 오존농도 0.033ppm 보통, 자외선지수 5 보통입니다.', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2769145560, 'id_str': '2769145560', 'name': '최준화', 'screen_name': 'junichoi94', 'location': '경기도 오산시', 'description': '현 WEB PROGRAMMER,\n현 AMATEUR PHOTOGRAPHER.\n현 IT & EDU. EXPERT FREELANCER.\n전 (주)토마토시스템 교육사업부 이사\n전 오산대학교 외래(시간) 강사\n전 대학교 정보전산원, 입학관리팀 근무\n전 법무부 교정위원 안양교도소 정보화교육 강사', 'url': 'https://t.co/8wZNpqagNV', 'entities': {'url': {'urls': [{'url': 'https://t.co/8wZNpqagNV', 'expanded_url': 'ht

In [125]:
# 사진도 같이 업로드!
sTime = []
sTime = datetime.datetime.now()

tweet = "[Juni]의 오늘(" + str(sTime) + ")의 날씨 안내 입니다.\n" + get_weather_data1()
media_ids = []
res = api.media_upload('./reporter.jpg') # 사진 업로드 먼저 하기!
media_ids.append(res.media_id)
api.update_status(media_ids=media_ids,status=tweet)

Status(_api=<tweepy.api.API object at 0x7f54d00bf3c8>, _json={'created_at': 'Thu Sep 24 09:51:56 +0000 2020', 'id': 1309068041003716613, 'id_str': '1309068041003716613', 'text': '[Juni]의 오늘(2020-09-24 09:51:54.915327)의 날씨 안내 입니다.\n경기도 오산시 궐동의 오늘 날씨는 오전 14˚, 오후 24˚, 체감온도 22.7˚, 미세먼지 10㎍/㎥ 좋음, 초미… https://t.co/GHFokduZcH', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/GHFokduZcH', 'expanded_url': 'https://twitter.com/i/web/status/1309068041003716613', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2769145560, 'id_str': '2769145560', 'name': '최준화', 'screen_name': 'junichoi94', 'location': '경기도 오산시', 'description': '현 WEB PROGRAMMER,\n현 AMATEUR PHOTOGRAPHER.\n현 IT & EDU. EXPERT FREELANCER.\n전 (주)토마토시스템 교육사업부 이사

# 과제 - 제한시간 15분 (바로 제가 이어서 같이 실행하도록 하겠습니다!)

## 트위터 날씨봇 만들기 ( 예시 : 매일 9시에 자동 업로드)

In [ ]:
def ~~ 어떤 함수를 써야할까요?

weather_url

In [117]:

import datetime
# 전체 코드
# 날씨 정보 가져오는 함수
def write_tweet():
    try:
        # 현재 날짜 정보 가져오기 (년,월,일)
        year = str(datetime.datetime.now().year)
        month = str(datetime.datetime.now().month)
        day = str(datetime.datetime.now().day)
        
        #  여기에 뭘 해야 할까요?
        # tweet = get_weather_data1("경기도 오산시 궐동")
        #

          # 트위터에 업로드
        #API Key 
        consumer_key = 'ExcRL3OPMPwVVOTx2VXNUvrQY'
        #API Key_Secret
        consumer_secret = 'wolrZNwPxIqqH9SJhZVjbrRFhzut3N388VKS7YCzgRdYuTog45'
        access_token = '2769145560-kLzZ2BSqj0xpwAUkS6aqBQ2qk6FgXVaamgWHXCZ'
        access_token_secret = 'IE8x6b5jqpZKF5DZspUHwaosP8k4iakEORAiwkGNhRJ4w'
        
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth)
        
        # 트윗내용
        tweet = get_weather_data1("경기도 오산시 궐동")
        media_ids = []
        res = api.media_upload('./reporter.jpg') #내가 원하는 다른 사진을 쓰셔도 됩니다.!
        media_ids.append(res.media_id)
        api.update_status(media_ids=media_ids,status=tweet)
        
    except:
        raise ValueError("에러났음")


In [119]:
write_tweet()

In [ ]:
!pip install schedule

# 자동화 설정

In [ ]:
####### 여기부터는 실행시키지 말아주세요!! #######

In [126]:
# 필요 패키지 가져오기
import schedule
from time import sleep

In [ ]:
# 매일 xx시 xx분 업로드 되게끔 설정
schedule.every().day.at("09:57").do(write_tweet)
while True:
    schedule.run_pending()
    sleep(30) 

In [ ]:
####### 여기까지는 실행시키지 말아주세요!! #######

In [122]:
datetime.datetime.now()

datetime.datetime(2020, 9, 24, 9, 38, 39, 477048)

## --------------------------------감사합니다----------------------------!!!!!!

![by-nc-nd.png](attachment:by-nc-nd.png)